In [ ]:
# Kaggle for dataset access
!pip install -q kaggle

# TensorFlow (already installed in Colab, but just in case)
!pip install -q tensorflow

# tqdm for progress bars
!pip install -q tqdm

# matplotlib for image visualization
!pip install -q matplotlib

# PIL for image processing (included in `Pillow`)
!pip install -q Pillow

!pip install opencv-python-headless

In [ ]:
from google.colab import files
files.upload()  # Upload your kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d chinafax/cfpw-dataset

In [ ]:
!unzip cfpw-dataset.zip -d dataset

In [ ]:
# Face Frontalization using G-GAN on CFPW Dataset (Colab-Compatible)

import os
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from PIL import Image
from tqdm import tqdm

# Set image size and constants
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 64
EPOCHS = 100

# Load and preprocess dataset
import glob
import random

def load_image_pair(profile_path, frontal_path):
    profile_img = Image.open(profile_path).resize((128, 128)).convert('RGB')
    frontal_img = Image.open(frontal_path).resize((128, 128)).convert('RGB')
    profile_img = ((np.array(profile_img).astype(np.float32)) / 127.5) - 1.0
    frontal_img = ((np.array(frontal_img).astype(np.float32)) / 127.5) - 1.0

    return profile_img, frontal_img

def load_dataset_pairs(base_path):
    profile_images = []
    frontal_images = []

    person_dirs = glob.glob(os.path.join(base_path, '*'))
    for person_dir in tqdm(person_dirs):
        profile_imgs = sorted(glob.glob(os.path.join(person_dir, 'profile', '*.jpg')))
        frontal_imgs = sorted(glob.glob(os.path.join(person_dir, 'frontal', '*.jpg')))

        min_len = min(len(profile_imgs), len(frontal_imgs))
        for i in range(min_len):
            prof, fron = load_image_pair(profile_imgs[i], frontal_imgs[i])
            profile_images.append(prof)
            frontal_images.append(fron)

    return np.array(profile_images), np.array(frontal_images)

# Load paired data
profile_imgs, frontal_imgs = load_dataset_pairs('/content/dataset/cfp-dataset/Data/Images')

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((profile_imgs, frontal_imgs)).shuffle(10000).batch(BATCH_SIZE)


# Build Generator (Encoder + Decoder)
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model

def build_generator(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape)
    x = inputs

    # Encoder with Dropout after some layers
    for i, filters in enumerate([16, 32, 64, 128, 256, 512,1024]):
        x = Conv2D(filters, kernel_size=4, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        if i >= 3:  # Apply dropout from the 4th layer onwards (128 and deeper)
            x = Dropout(0.3)(x)

    # Decoder with Dropout after some layers
    for i, filters in enumerate([ 512, 256, 128, 64, 32, 16]):
        x = Conv2DTranspose(filters, kernel_size=4, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        if i < 3:  # Apply dropout to first few decoder layers (closest to bottleneck)
            x = Dropout(0.3)(x)

    # Final output layer
    x = Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh')(x)
    return Model(inputs, x, name="G_GAN_Generator_With_Dropout")

# Build Discriminator

def build_discriminator(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape)
    x = inputs
    for filters in [16, 32, 64, 128, 256, 512]:
        x = Conv2D(filters, kernel_size=3, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs, x, name="G_GAN_Discriminator")

# Define custom loss function
def combined_gan_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    l1 = tf.reduce_mean(tf.square(y_true - y_pred))  # MSE
    l2 = tf.reduce_mean(tf.abs(y_true - y_pred))     # MAE
    l3 = tf.reduce_mean(tf.square(y_true - y_pred))  # Again MSE, per paper's formula

    return 0.001 * l1 + 1.0 * l2 + 1.0 * l3

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Optimizers
g_optimizer = Adam(0.0002, beta_1=0.5)
d_optimizer = Adam(0.0002, beta_1=0.5)

# Binary cross-entropy loss
bce = tf.keras.losses.BinaryCrossentropy()

# Training step
@tf.function
def train_step(profile_images, real_frontal_images):
    with tf.GradientTape(persistent=True) as tape:
        generated_frontal = generator(profile_images, training=True)

        real_output = discriminator(real_frontal_images, training=True)
        fake_output = discriminator(generated_frontal, training=True)

        d_loss_real = bce(tf.ones_like(real_output), real_output)
        d_loss_fake = bce(tf.zeros_like(fake_output), fake_output)
        d_loss = d_loss_real + d_loss_fake

        g_loss_gan = bce(tf.ones_like(fake_output), fake_output)
        g_loss_custom = combined_gan_loss(real_frontal_images, generated_frontal)
        g_total_loss = g_loss_gan + g_loss_custom

    gradients_of_generator = tape.gradient(g_total_loss, generator.trainable_variables)
    gradients_of_discriminator = tape.gradient(d_loss, discriminator.trainable_variables)

    g_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    d_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return d_loss, g_total_loss

# Training loop
def train(dataset, epochs):
    best_g_loss = float('inf')

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for profile_batch, frontal_batch in dataset:
            d_loss, g_loss = train_step(profile_batch, frontal_batch)

        print(f"Discriminator Loss: {d_loss.numpy():.4f}, Generator Loss: {g_loss.numpy():.4f}")

        # Save best model based on generator loss
        if g_loss < best_g_loss:
            best_g_loss = g_loss
            generator.save('/content/drive/MyDrive/GGAN_Models_E400/best_generator.h5')
            print("🔁 Saved best generator (g_loss improved).")



# Save sample images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0
    fig = plt.figure(figsize=(8, 2))
    for i in range(predictions.shape[0]):
        plt.subplot(1, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')
    plt.savefig(f"image_at_epoch_{epoch}.png")
    plt.close()

# Start training
train(dataset, EPOCHS)


In [ ]:
import matplotlib.pyplot as plt

# Set generator to evaluation mode (in Keras, this is done implicitly)
def visualize_generated_images(generator, profile_images, num_samples=4):
    profile_images = profile_images[:num_samples]

    generated_images = generator(profile_images, training=False)
    generated_images = (generated_images + 1.0) / 2.0  # De-normalize to [0, 1]
    profile_images = (profile_images + 1.0) / 2.0

    plt.figure(figsize=(num_samples * 2, 4))

    for i in range(num_samples):
        # Show original profile image
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(profile_images[i])
        plt.title("Profile")
        plt.axis('off')

        # Show generated frontal image
        plt.subplot(2, num_samples, i + 1 + num_samples)
        plt.imshow(generated_images[i])
        plt.title("Generated Front")
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Sample 4 test images from dataset
sample_profiles = profile_imgs[:4]

# Visualize results
visualize_generated_images(generator, sample_profiles, num_samples=4)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create a directory in your Drive (once)
save_path = '/content/drive/MyDrive/GGAN_Models'
os.makedirs(save_path, exist_ok=True)

# Save generator
generator.save(os.path.join(save_path, 'ggan_generator.h5'))

# Optional: Save discriminator
discriminator.save(os.path.join(save_path, 'ggan_discriminator.h5'))